In [1]:
import tkinter as tk
from tkinter import *
import tkinter.messagebox
import pyaudio
import wave
import os
import numpy as np
import math
import operator

In [2]:
CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 2
RATE = 44100
WAVE_OUTPUT_FILENAME = "output/voice.wav"

In [6]:
p = pyaudio.PyAudio()
recording = 0
frames = []
index = 0

In [7]:
import pickle
with open("modelHMM.pkl", "rb") as file:
    models = pickle.load(file)

In [8]:
import functionMFCC as func

In [9]:
stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)

In [15]:
text_predicted = {'cua': 'của', 'viet_nam': 'Việt Nam', 'khong': 'không', 'nguoi': 'người', 'nha': 'nhà'}
def predict_voice():
    inp = func.get_mfcc(os.path.join("output", "voice.wav"))
    score = {cname: model.score(inp, [len(inp)]) for cname, model in models.items()}
    print(score)
    text = max([(value, key) for key, value in score.items()])[1]
    if text == 'cua' and score['cua']-score['khong']<500:
        text = 'khong'
    outputBox.delete("1.0", END)
    outputBox.insert(END, text_predicted[text])

In [11]:
def start_record():
    recording = 1
    frames.clear()
    label['text'] = 'Đang ghi âm'
    stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)
    while recording == 1:
        data = stream.read(CHUNK)
        frames.append(data)
        window.update()

In [12]:
def stop_record():
    recording = 0
    label['text'] = 'Dự đoán là'
    print('* stop recording')
    stream.stop_stream()
    stream.close()
    wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()
    predict_voice()

In [13]:
def play_wav():
    p_read = pyaudio.PyAudio()
    wf_read = wave.open(os.path.join("output", "voice.wav"), 'rb')
    stream_read = p_read.open(format =
                            p_read.get_format_from_width(wf_read.getsampwidth()),
                            channels = wf_read.getnchannels(),
                            rate = wf_read.getframerate(),
                            output = True)
    data = wf_read.readframes(CHUNK)
    while data != '':
        # writing to the stream is what *actually* plays the sound.
        stream_read.write(data)
        data = wf_read.readframes(CHUNK)
        window.update()
    stream_read.close()    
    p_read.terminate()

In [16]:
window = tk.Tk()
window.geometry('500x200')
window.title('Predict voice')
button_start = tk.Button(window, text='Start', width=15, command=start_record)
button_start.place(x=70, y=50)
button_stop = tk.Button(window, text='Stop', width=15, command=stop_record)
button_stop.place(x=190, y=50)
button_play = tk.Button(window, text='Play', width=15, command=play_wav)
button_play.place(x=310, y=50)
label = tk.Label(window)
label.place(x=230, y=100)
outputBox = Text(window, width=30, height=1, bd=1)
outputBox.place(x=150, y=130)
window.mainloop()

Degenerate mixture covariance


* stop recording
{'khong': -8303.171304048194, 'nha': -7497.858282499057, 'cua': -7877.820185396362, 'nguoi': -8008.82890306568, 'viet_nam': -7848.798881441874}


Degenerate mixture covariance


* stop recording
{'khong': -7478.492289518903, 'nha': -7703.72303801121, 'cua': -7352.101002978321, 'nguoi': -7736.3547017399815, 'viet_nam': -7898.090686367277}


Degenerate mixture covariance


...

Degenerate mixture covariance


* stop recording
{'khong': -9166.379331438015, 'nha': -9961.870278900513, 'cua': -9488.70839602149, 'nguoi': -9821.231334212354, 'viet_nam': -10041.12951657892}


Degenerate mixture covariance


* stop recording
{'khong': -8864.862547023762, 'nha': -8789.864805759928, 'cua': -7629.668934954127, 'nguoi': -8946.207944868107, 'viet_nam': -8628.123121716762}


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\PhuongDong\miniconda3\envs\voice\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-11-91ae551fdff6>", line 13, in start_record
    window.update()
  File "C:\Users\PhuongDong\miniconda3\envs\voice\lib\tkinter\__init__.py", line 1177, in update
    self.tk.call('update')
_tkinter.TclError: can't invoke "update" command: application has been destroyed
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\PhuongDong\miniconda3\envs\voice\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-11-91ae551fdff6>", line 13, in start_record
    window.update()
  File "C:\Users\PhuongDong\miniconda3\envs\voice\lib\tkinter\__init__.py", line 1177, in update
    self.tk.call('update')
_tkinter.TclError: can't invoke "update" command: application has been destroyed
Exception in Tkinter callback
Tr